Code to get the latest reading from a sesor: (https://preprodapi.mde.epf.fr/measure.php?id=33&row_count=1)

Local Url: (http://192.168.139.27/measure.php?id=33&createdLenght=13&row_count=1)


### CO2 Sensor ids: 19, 33, 40, 43, 46, 227

| Sensor id| Condition | Position |
|----------|----------|----------|
| 19    | Not Working  | NA   |
| 33    | Working  | To Do  |
| 40    | Working   | TO Do   |
| 43    | Working   | TO Do   |
| 46    | Working   | TO Do   |
| 227    | Working   |TO Do   |

#### **Using**
| Sensor id| Condition | Position |
|----------|----------|----------|
| 33    | Working  | To Do  |

### Sound Sensor ids: 32

| Sensor id| Condition | Position |
|----------|----------|----------|
| 32    | Working  | To Do  |

### Javascript output from the webpage

```Javascript
[
    {
        "general": {
            "id_system_sensor": "32",
            "name": "EL_NA_7fa542_0101",
            "description": "Plateau Principal Bruit\r\n",
            "unite": "32",
            "project": {
                "id_project": "17",
                "name": "EL_NA_7fa542",
                "description": "ENERGYLAB_NETATMO_PP",
                "project": {
                    "id_project": "16",
                    "name": "EL_NA",
                    "description": "ENERGYLAB_NETATMO",
                    "project": {
                        "id_project": "1",
                        "name": "EL",
                        "description": "ENERGYLAB"
                    }
                }
            }
        }
    },
    {
        "created": "2024-10-17 09:12:15",
        "value": "56",
        "id_system_sensor": "32"
    }
]

```

In [ ]:
import requests
import json

# List of sensor IDs
sensor_ids = [32, 33]

# Updated base URL for the API
base_url = "http://192.168.139.27/measure.php"

# Dictionary to store the results for JavaScript output
sensor_data = {}

# Sensor names
sensor_names = {
    32: "sound",           
    33: "Co2",
}

# Loop through each sensor ID and get the final reading
for sensor_id in sensor_ids:
    # Set the parameters to get the latest reading for each sensor
    params = {
        'id': sensor_id,
        'row_count': 1  # Fetch the most recent reading
    }
    
    # Make the request to the updated API URL
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Extract the sensor ID, the latest value, and the sensor name
        sensor_data[sensor_id] = {
            "sensor": sensor_names[sensor_id],  # Add the corresponding sensor name
            "id_system_sensor": data[0]["general"]["id_system_sensor"],
            "value": data[1]["value"]
        }
    else:
        print(f"Failed to retrieve data for Sensor ID {sensor_id}")

# Create JavaScript file content
js_content = "// Sensor Data\nvar sensorData = [\n"
for sensor in sensor_data.values():
    js_content += f'  {{\n'
    js_content += f'    "sensor": "{sensor["sensor"]}",\n'
    js_content += f'    "id_system_sensor": "{sensor["id_system_sensor"]}",\n'
    js_content += f'    "value": "{sensor["value"]}"\n'
    js_content += f'  }},\n'
js_content = js_content.rstrip(",\n") + "\n];"  # Remove the last comma and close the array

# Write the content to a JavaScript file
with open("latest_sensorData.js", "w") as js_file:
    js_file.write(js_content)

print("JavaScript file 'sensorData.js' created successfully.")


JavaScript file 'sensorData.js' created successfully.


## Extract data to a .csv

In [2]:
import requests
import csv

# List of sensor IDs
sensor_ids = [32, 33]

# Sensor names corresponding to IDs
sensor_names = {
    32: "sound",
    33: "Co2",
}

# Base URL for the API
base_url = "http://192.168.139.27/measure.php"

# Number of rows to retrieve (or fetch all available data if API allows)
row_count = 120000 # Row Count

# Dictionary to hold data for each sensor
sensor_data = {
    32: [],  # For sound sensor
    33: []   # For CO2 sensor
}

# Loop through each sensor and fetch the data
for sensor_id in sensor_ids:
    # Set parameters to fetch all the data for each sensor
    params = {
        'id': sensor_id,
        'row_count': row_count  # Specify the number of records to fetch
    }

    # Make the request to the API
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Adjust extraction based on actual response format
        for entry in data:
            if "created" in entry and "value" in entry:
                sensor_data[sensor_id].append({
                    "time": entry["created"],  # Timestamp of the reading
                    "value": entry["value"]    # Sensor value
                })
            else:
                print(f"Skipping entry without 'created' or 'value': {entry}")
    else:
        print(f"Failed to retrieve data for Sensor ID {sensor_id}")

# Write the data to a CSV file
with open("raw_sensor_data.csv", mode='w', newline='') as csv_file:
    fieldnames = ["time", "sound_value", "co2_value"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Assuming both sensors have the same number of records, iterate over them
    for i in range(min(len(sensor_data[32]), len(sensor_data[33]))):
        writer.writerow({
            "time": sensor_data[32][i]["time"], 
            "sound_value": sensor_data[32][i]["value"],
            "co2_value": sensor_data[33][i]["value"]
        })

print("CSV file 'sensor_data.csv' created successfully.")


Skipping entry without 'created' or 'value': {'general': {'id_system_sensor': '32', 'name': 'EL_NA_7fa542_0101', 'description': 'Plateau Principal Bruit\r\n', 'unite': '32', 'project': {'id_project': '17', 'name': 'EL_NA_7fa542', 'description': 'ENERGYLAB_NETATMO_PP', 'project': {'id_project': '16', 'name': 'EL_NA', 'description': 'ENERGYLAB_NETATMO', 'project': {'id_project': '1', 'name': 'EL', 'description': 'ENERGYLAB'}}}}}
Skipping entry without 'created' or 'value': {'general': {'id_system_sensor': '33', 'name': 'EL_NA_7fa542_0100', 'description': 'Plateau Principal CO2\r\n', 'unite': '33', 'project': {'id_project': '17', 'name': 'EL_NA_7fa542', 'description': 'ENERGYLAB_NETATMO_PP', 'project': {'id_project': '16', 'name': 'EL_NA', 'description': 'ENERGYLAB_NETATMO', 'project': {'id_project': '1', 'name': 'EL', 'description': 'ENERGYLAB'}}}}}
CSV file 'sensor_data.csv' created successfully.
